In [283]:
'''Current issues for all functions:
    Fair Price Error (Make it so that is bid or ask is 0, fair price is 0.01)
    Get float to round upto 2 decimal places
    Make the entire thing wayyy more efficient
'''
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
import numpy as np
import yfinance as yf
import time
from os import system, name

s = yf.Ticker("MMYT")
opt = s.option_chain('2020-08-20')
calls, puts = opt.calls, opt.puts
print(int(calls['volume'].mean()))
calls['Fair Price'] = [i if i >= 0.01 else 0.01 for i in (calls['ask']+calls['bid'])/2]
puts['Fair Price']  = [i if i >= 0.01 else 0.01 for i in (puts['ask'] + puts['bid'])/2]

11


In [284]:
def Call_Credit_Spread():
    
    c = calls[calls['volume']>=int(calls['volume'].mean())]
    l = c[['strike', 'Fair Price']]
    j = c[['strike', 'lastPrice']]
    if c['bid'].mean() != 0:
        m = l.set_index('strike').to_dict()
        n = m['Fair Price'] 
        print("Using Current Market Price")
    else:
        m = j.set_index('strike').to_dict()
        n = m['lastPrice']
        print("Using Last Price")
    
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

# best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}

    # First loop to check what to short (sell)   
    for short in n:
        
        # Second loop to check what we're buying (long)
        for long in n:
            
            # In a credit call spread the long Strike cannot be lower or equal to the short strike price
            if long<=short:
                continue
            else:
                
                # Declaring variables to work with
                strikeShort = short
                strikeLong = long
                
                premiumShort = n[short]
                premiumLong = n[long]
                
                # Figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = premiumShort - premiumLong
                
                #print("MaxReward = ", MaxReward)
                
                MaxRisk = (strikeLong - strikeShort - MaxReward)
                
                #print("MaxRisk = ", MaxRisk)
                
                # Cannot divide by 0.
                if MaxReward == 0:
                    continue
                
                # We don't want the reward to be in negative, which means that no matter what, we won't be profitting
                elif MaxReward>0:
                    
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
    # returning the final best trade available as a call credit spread              
    return print("Best Call Credit Spread: ", best_ratio)
        



In [285]:
def Put_Credit_Spread():
    
    p = puts[puts['volume']>=int(puts['volume'].mean())]
    l = p[['strike', 'Fair Price']]
    j = p[['strike', 'lastPrice']]
    if p['bid'].mean() != 0:
        m = l.set_index('strike').to_dict()
        n = m['Fair Price'] 
        print("Using Current Market Price")
    else:
        m = j.set_index('strike').to_dict()
        n = m['lastPrice']
        print("Using Last Price")
    
#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

    # best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    
    # First loop to check what to long (buy) 
    for long in n:
        
        # Second loop to check what we're selling(short)
        for short in n:
            
            # In a put credit spread the strike price of what we're buying cannot be higher than strike price of what we're selling
            if long >= short:
                continue
                
            else:
                
                #Decalring various variables
                strikeShort = short
                strikeLong = long
                premiumShort = n[short]
                premiumLong = n[long]
                
                #figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = premiumShort - premiumLong      
                #print("MaxReward = ", MaxReward)
                
                MaxRisk = (strikeLong - strikeShort - MaxReward)
                MaxRisk*=-1
                #print("MaxRisk = ", MaxRisk)
                
                # Cannot divide by 0.
                if MaxReward == 0:
                    continue
                    
                # Don't want negative Rewards...  
                elif MaxReward>0:
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
                        
    return print("Best Put Credit Spread: ", best_ratio)

In [286]:
def Put_Debit_Spread():
    
    p = puts[puts['volume']>=int(puts['volume'].mean())]
    l = p[['strike', 'Fair Price']]
    j = p[['strike', 'lastPrice']]
    if p['bid'].mean() != 0:
        m = l.set_index('strike').to_dict()
        n = m['Fair Price'] 
        print("Using Current Market Price")
    else:
        m = j.set_index('strike').to_dict()
        n = m['lastPrice']
        print("Using Last Price")

#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio

    # best_ratio: Dictionary to maintain the best available trade at a given time
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    
    # First loop to check what we're buying
    for long in n:
        
        # Second loop for what we're selling
        for short in n:
            
            # We don't want buying strike to be smaller than selling strike (Put debit, buy higher strike and sell lower strike)
            if long <= short:
                continue
            else:
                
                #Declaring variables
                strikeShort = short
                strikeLong = long
                premiumShort = n[short]
                premiumLong = n[long]
                
                # figuring out the max risk vs reward in each spread to minimise the risk/reward ratio.
                MaxReward = strikeLong - strikeShort - premiumLong - premiumShort
                MaxRisk = premiumLong - premiumShort
                
                # Cannot divide by 0.
                if MaxReward == 0:
                    continue
                    
                elif MaxReward>0:
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
                        
    return print("Best Put Debit Spread: ", best_ratio)

In [287]:
#Put_Debit_Spread()

In [288]:
def Call_Debit_Spread():
    
    c = calls[calls['volume']>=int(calls['volume'].mean())]
    l = c[['strike', 'Fair Price']]
    j = c[['strike', 'lastPrice']]
    if c['bid'].mean() != 0:
        m = l.set_index('strike').to_dict()
        n = m['Fair Price'] 
        print("Using Current Market Price")
    else:
        m = j.set_index('strike').to_dict()
        n = m['lastPrice']
        print("Using Last Price")

#     We need to check the risk, reward, break even point
#     Then we need to check the ratio of the risk/reward
#     Then we need to store the 'Transaction' of the least ratio
    best_ratio = {'Short(Sell)':0, 'Long(Buy)':0, 'Risk/Reward Ratio':((2**31)-1), 'Short Premium':0, 'Long Premium':0, 'maxRisk':0, 'maxReward':0}
    
    
    # First loop to check what we're buying
    for long in n:
        
        # Second loop for what we're selling
        for short in n:
            
            # We don't want buying strike to be higher than selling strike (Call debit, sell higher strike and buy lower strike)
            if long >= short:
                continue
            else:
                
                #Declaring variables
                strikeShort = short
                strikeLong = long
                premiumShort = n[short]
                premiumLong = n[long]
                

                MaxRisk = premiumLong - premiumShort
                MaxReward = strikeLong - strikeShort - MaxRisk
                MaxReward*=-1
                
                if MaxReward == 0:
                    continue
                    
                    
                elif MaxReward>0:
                    Ratio =  MaxRisk/MaxReward
                    if Ratio < best_ratio['Risk/Reward Ratio']:
                        best_ratio['Long(Buy)']=long
                        best_ratio['Short(Sell)']=short
                        best_ratio['Risk/Reward Ratio']=Ratio
                        best_ratio['Short Premium'] = n[short]
                        best_ratio['Long Premium'] = n[long]
                        best_ratio['maxRisk'] = MaxRisk
                        best_ratio['maxReward'] = MaxReward
                        
                        
    return print("Best Call Debit Spread: ", best_ratio)

In [289]:
def finalSpreads():
    print("\t\t\t~~~~~~~~~~~~~~~~~~~~~~~~Bullish~~~~~~~~~~~~~~~~~~~~~~~~~ \n")
    Call_Debit_Spread()
    print("\n")
    Put_Credit_Spread()
    print("\n")
    print("\t\t\t~~~~~~~~~~~~~~~~~~~~~~~~Bearish~~~~~~~~~~~~~~~~~~~~~~~~~ \n")
    Put_Debit_Spread()
    print("\n")
    Call_Credit_Spread()

In [290]:
finalSpreads()

			~~~~~~~~~~~~~~~~~~~~~~~~Bullish~~~~~~~~~~~~~~~~~~~~~~~~~ 

Using Current Market Price
Best Call Debit Spread:  {'Short(Sell)': 35.0, 'Long(Buy)': 20.0, 'Risk/Reward Ratio': -0.1583011583011583, 'Short Premium': 2.5, 'Long Premium': 0.45, 'maxRisk': -2.05, 'maxReward': 12.95}


Using Current Market Price
Best Put Credit Spread:  {'Short(Sell)': 12.5, 'Long(Buy)': 7.5, 'Risk/Reward Ratio': 9.695652173913045, 'Short Premium': 0.85, 'Long Premium': 0.275, 'maxRisk': 5.575, 'maxReward': 0.575}


			~~~~~~~~~~~~~~~~~~~~~~~~Bearish~~~~~~~~~~~~~~~~~~~~~~~~~ 

Using Current Market Price
Best Put Debit Spread:  {'Short(Sell)': 5.0, 'Long(Buy)': 12.5, 'Risk/Reward Ratio': -0.406060606060606, 'Short Premium': 2.525, 'Long Premium': 0.85, 'maxRisk': -1.6749999999999998, 'maxReward': 4.125}


Using Current Market Price
Best Call Credit Spread:  {'Short(Sell)': 17.5, 'Long(Buy)': 20.0, 'Risk/Reward Ratio': 6.692307692307692, 'Short Premium': 0.775, 'Long Premium': 0.45, 'maxRisk': 2.175, 'maxRewar